In [ ]:
# import libraries
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)
pd.set_option('display.max_columns', 50)
import sqlite3
import matplotlib.dates as mdates
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
from bokeh.models import HoverTool
from IPython.display import HTML, display

In [ ]:
con = sqlite3.connect("dataset.sqlite3")
df = pd.read_sql_query('''SELECT authors.field,authors.sub_field,authors.scholar_id,
                        publication_url,publication_year, total_authors 
                        FROM publications_computed_fields join authors
                        on publications_computed_fields.scholar_id=authors.scholar_id where publication_year<2024 and publication_year is not NULL''', con)

con.close()

In [ ]:
#finding consecutive years data
def getcontinuousData(df):
    all_years = sorted(df['publication_year'].unique())

    max_continuous_years = []
    current_continuous_years = []

    for year in all_years:
        if not current_continuous_years or year == current_continuous_years[-1] + 1:
            current_continuous_years.append(year)
        else:
            if len(current_continuous_years) > len(max_continuous_years):
                max_continuous_years = current_continuous_years
            current_continuous_years = [year]

    if len(current_continuous_years) > len(max_continuous_years):
        max_continuous_years = current_continuous_years
    
    # If consecutive years are found, select the corresponding portion of the DataFrame
    if max_continuous_years:
        start_year = max_continuous_years[0]
        end_year = max_continuous_years[-1]
        selected_data = df[(df['publication_year'] >= start_year) & (df['publication_year'] <= end_year)].sort_values(by=['publication_year'], ascending=True)

        # print(selected_data['publication_year'])
        print(start_year,end_year)
        return selected_data
    else:
        print("No consecutive years found.")
        return None

In [ ]:
sub_fields = df['sub_field'].unique()

In [ ]:
result_list = []

for sub_field in sub_fields:
    temp_df = df[(df['sub_field']==sub_field)]
    temp_df = getcontinuousData(temp_df)
    start_year = temp_df['publication_year'].min()
    cnt = temp_df[(temp_df['publication_year']==1980)]['publication_url'].count()
    print(sub_field+"-----------------Year----"+str(1980)+"---publication count---"+str(cnt))
    if cnt < 8:
        result_list.append({"sub_field": sub_field, "Year": 1980, "cnt": cnt})
result_df = pd.DataFrame(result_list)
# Save the result to a CSV file
result_df.to_csv("publication_counts_1980_less_than_8.csv", index=False)

print("Results saved to publication_counts_1980.csv")
    # print(temp_df[(temp_df['publication_year']>=1977)].groupby('publication_year')['publication_url'].count())

In [ ]:
fields = df['field'].unique()

In [ ]:
for field in fields:
    temp_df = df[(df['field']==field)]
    temp_df = getcontinuousData(temp_df)
    start_year = temp_df['publication_year'].min()
    cnt = temp_df[(temp_df['publication_year']==1980)]['publication_url'].count()
    print(field+"-----------------Year----"+str(1980)+"---publication count---"+str(cnt))
    # print(temp_df[(temp_df['publication_year']>=1977)].groupby('publication_year')['publication_url'].count())

In [ ]:
fields = df['field'].unique()
result_list = []
min_lim = 10
for field in fields:
    temp_df = df[(df['field']==field)]
    filtered_df = temp_df.groupby('publication_year').filter(lambda x: x['publication_url'].count() >= min_lim)
    filtered_df = getcontinuousData(filtered_df)
    start_year = filtered_df['publication_year'].min()
    cnt = filtered_df[(filtered_df['publication_year']==start_year)]['publication_url'].count()
    print(field+"-----------------Year----"+str(start_year)+"---publication count---"+str(cnt))
    result_list.append({"field": field, "year": start_year, "publication_count": cnt})
    # print(temp_df[(temp_df['publication_year']>=1977)].groupby('publication_year')['publication_url'].count())

result_df = pd.DataFrame(result_list)
result_df.to_csv("publication_counts_field_min_lim_"+str(min_lim)+".csv", index=False)

print("Results saved")


# filtered_df = df.groupby('publication_year').filter(lambda x: x['publication_url'].count() >= 10)

# filtered_df = getcontinuousData(filtered_df)

# temp_df = filtered_df.groupby('publication_year')['publication_url'].count()

# print(temp_df.index[0],temp_df.values[0])

In [ ]:
sub_fields = df['sub_field'].unique()
min_lim = 5

result_list = []
for sub_field in sub_fields:
    temp_df = df[(df['sub_field']==sub_field)]
    filtered_df = temp_df.groupby('publication_year').filter(lambda x: x['publication_url'].count() >= min_lim)
    filtered_df = getcontinuousData(filtered_df)
    start_year = filtered_df['publication_year'].min()
    cnt = filtered_df[(filtered_df['publication_year']==start_year)]['publication_url'].count()
    print(sub_field+"-----------------Year----"+str(start_year)+"---publication count---"+str(cnt))
    result_list.append({"sub_field": sub_field, "year": start_year, "publication_count": cnt})
    # print(temp_df[(temp_df['publication_year']>=1977)].groupby('publication_year')['publication_url'].count())
result_df = pd.DataFrame(result_list)
result_df.to_csv("publication_counts_sub_field_min_lim_"+str(min_lim)+".csv", index=False)

In [ ]:
fields = df['field'].unique()
result_list = []
for field in fields:
    filtered_df = df[(df['field']==field)]
    filtered_df = getcontinuousData(filtered_df)
    start_year = 1980
    cnt = filtered_df[(filtered_df['publication_year']==start_year)]['publication_url'].count()
    print(field+"-----------------Year----"+str(start_year)+"---publication count---"+str(cnt))
    result_list.append({"field": field, "year": start_year, "publication_count": cnt})
    # print(temp_df[(temp_df['publication_year']>=1977)].groupby('publication_year')['publication_url'].count())

result_df = pd.DataFrame(result_list)
result_df.to_csv("publication_counts_field_1980.csv", index=False)

print("Results saved")

In [ ]:
result_list = []
years_to_check = [1977, 1978, 1979, 1980, 1981, 1982,1983,1984,1985,1986,1987]

for sub_field in sub_fields:
    temp_df = df[df['sub_field'] == sub_field]
    temp_df = getcontinuousData(temp_df)
    counts = {'sub_field': sub_field}
    for year in years_to_check:
        cnt = temp_df[temp_df['publication_year'] == year]['publication_url'].count()
        counts[year] = cnt
        print(f"{sub_field}-----------------Year----{year}---publication count---{cnt}")
    # if any(cnt < 8 for cnt in counts.values()):
    result_list.append(counts)

result_df = pd.DataFrame(result_list)
# Save the result to a CSV file
result_df.to_csv("publication_counts_1977_1987.csv", index=False)

print("Results saved to publication_counts_1977_1987.csv")

In [ ]:
con = sqlite3.connect("dataset.sqlite3")
collection_df = pd.read_sql_query('''SELECT authors.field, authors.sub_field,count(publications.publication_url) as count
                        FROM authors left join publications on authors.scholar_id=publications.scholar_id
                        group by authors.sub_field,authors.field''', con)

con.close()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))

# Create a bar chart
for field in collection_df['field'].unique():
    subset = collection_df[collection_df['field'] == field]
    ax.barh(subset['sub_field'], subset['count'], label=field)

# Adding labels and title
ax.set_ylabel('Sub Fields')
ax.set_xlabel('Number of Publications')
plt.yticks(fontsize = 8)
ax.set_title('Number of Publications Collected For Each Sub Field')
ax.legend(title='Fields')

# Display the plot
plt.show()


In [ ]:
con = sqlite3.connect("dataset.sqlite3")
collection_df = pd.read_sql_query('''SELECT authors.field, authors.sub_field,count(scholar_id) as count
                        FROM authors
                        group by authors.sub_field,authors.field''', con)

con.close()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))

# Create a bar chart
for field in collection_df['field'].unique():
    subset = collection_df[collection_df['field'] == field]
    ax.barh(subset['sub_field'], subset['count'], label=field)

# Adding labels and title
ax.set_ylabel('Sub Fields')
ax.set_xlabel('Number of Author Profile')
plt.yticks(fontsize = 8)
ax.set_title('Number of Profiles Collected For Each Sub Field')
ax.legend(title='Fields')
plt.savefig('author_profile_by_sub_field.png')
# Display the plot

# Display the plot
plt.show()


In [ ]:
con = sqlite3.connect("dataset.sqlite3")
collection_df_author_field = pd.read_sql_query('''SELECT authors.field,count(scholar_id) as count
                        FROM authors group by authors.field''', con)

con.close()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))

# Create a bar chart
for field in collection_df_author_field['field'].unique():
    subset = collection_df_author_field[collection_df_author_field['field'] == field]
    ax.barh(subset['field'], subset['count'], label=field)

# Adding labels and title
ax.set_ylabel('Fields')
ax.set_xlabel('Number of Author Profile')
plt.yticks(fontsize = 8)
ax.set_title('Number of Author Profile Collected For Each Field')
ax.legend(title='Fields')
plt.savefig('author_profile_by_field.png')
# Display the plot
plt.show()
